In [ ]:
#一键检验稳定性
import time
import json
import requests
import pandas as pd
import numpy as np
import requests
from requests.auth import HTTPBasicAuth
from urllib.request import urlopen


alpha_id_ori='dJ8RAGx'


def locate_alpha(s, alpha_id):
    while True:
        alpha = s.get("https://api.worldquantbrain.com/alphas/" + alpha_id)
        if "retry-after" in alpha.headers:
            time.sleep(float(alpha.headers["Retry-After"]))
        else:
            break
    string = alpha.content.decode('utf-8')
    metrics = json.loads(string)
    #print(metrics["regular"]["code"])
    
    dateCreated = metrics["dateCreated"]
    sharpe = metrics["is"]["sharpe"]
    fitness = metrics["is"]["fitness"]
    turnover = metrics["is"]["turnover"]
    margin = metrics["is"]["margin"]
    decay = metrics["settings"]["decay"]
    exp = metrics['regular']['code']
    universe=metrics["settings"]["universe"]
    truncation=metrics["settings"]["truncation"]
    neutralization=metrics["settings"]["neutralization"]
    region=metrics["settings"]["region"]

    # triple =pd.DataFrame([[alpha_id,  sharpe, turnover, fitness, margin]])
    triple =[alpha_id,  sharpe, turnover, fitness, margin, exp,region,universe,neutralization,decay,truncation]
    return triple


def login():

   
    username = ""
    password = ""

 
    # Create a session to persistently store the headers
    s = requests.Session()
 
    # Save credentials into session
    s.auth = (username, password)
 
    # Send a POST request to the /authentication API
    response = s.post('https://api.worldquantbrain.com/authentication')
    print(response.content)
    return s  

s = login()
sharp_list = []
df=pd.DataFrame(columns=['alpha_id','sharpe','turnover','fitness','margin'])




alpha_line = []
tem=locate_alpha(s, alpha_id_ori)

[alpha_id,  sharpe, turnover, fitness, margin, exp,region,universe,neutralization,decay,truncation]=tem



# 根据 decay 的值选择不同的 decay_tem 列表
decay_tem_list = [decay - 5, decay + 5] if decay >= 5 else [decay + 10, decay + 20]
decay_tem_list.append(decay)

# 固定的 neutralization_tem 列表
neutralization_tem_list = ['SUBINDUSTRY', 'INDUSTRY', 'SECTOR', 'MARKET', 'CROWDING','SLOW_AND_FAST', 'REVERSION_AND_MOMENTUM', 'STATISTICAL', 'SLOW', 'FAST']

# neutralization_tem_list = ['SUBINDUSTRY']
# 使用列表推导式生成 simulation_data
alpha_line.extend(
    {
        'type': 'REGULAR',
        'settings': {
            'instrumentType': 'EQUITY',
            'region': region,
            'universe': universe,
            'delay': 1,
            'decay': decay_tem,
            'neutralization': neutralization_tem,
            'truncation': truncation,
            'pasteurization': 'ON',
            'unitHandling': 'VERIFY',
            'nanHandling': 'ON',
            'language': 'FASTEXPR',
            'visualization': False,
            'testPeriod': "P0Y",
            'maxTrade': 'OFF'
        },
        'regular': exp
    }
    for decay_tem in decay_tem_list
    for neutralization_tem in neutralization_tem_list
)


                    
print(len(alpha_line))
wbs = []

batch_size = 10

for i in range(0, len(alpha_line), batch_size):
    alpha_batch = alpha_line[i:i + batch_size]
    print(alpha_batch)
    print(len(alpha_batch))
    retry_count = 0

    while True:
        try:
            # Send one request with 10 alpha_data as a list
            response = s.post('https://api.worldquantbrain.com/simulations', json=alpha_batch)

            # 检查 HTTP 状态码，如果失败（例如 4xx 或 5xx），抛出异常
            response.raise_for_status()
            # Assume response contains a list of "Location" headers for each alpha
            # 检查响应头中是否包含 Location 字段
            if "Location" in response.headers:
                # 如果成功获取 Location，记录日志并返回
                print("Alpha batch location retrieved successfully.")
                print(f"Location: {response.headers['Location']}")
                wbs.append(response.headers['Location'])
                break
        except Exception as e:
            retry_count += 1
            print(f"Error sending batch simulation request. Retry {retry_count} after 1min. Error: {e}")
            time.sleep(60)

            if retry_count > 25:
                print("Retry >25: attempting re-login.")
                s = login()
            if retry_count > 30:
                print("Retry >30: giving up on this batch.")
                break

print(len(wbs))


b'{"user":{"id":"YT33830"},"token":{"expiry":14400.0},"permissions":["BEFORE_AND_AFTER_PERFORMANCE_V2","BRAIN_LABS","BRAIN_LABS_JUPYTER_LAB","CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","SUPER_ALPHA","VISUALIZATION","WORKDAY"]}'
30
[{'type': 'REGULAR', 'settings': {'instrumentType': 'EQUITY', 'region': 'EUR', 'universe': 'TOP2500', 'delay': 1, 'decay': 1, 'neutralization': 'SUBINDUSTRY', 'truncation': 0.08, 'pasteurization': 'ON', 'unitHandling': 'VERIFY', 'nanHandling': 'ON', 'language': 'FASTEXPR', 'visualization': False, 'testPeriod': 'P0Y', 'maxTrade': 'OFF'}, 'regular': 'group_zscore(-(ts_zscore(group_rank(winsorize(ts_backfill(vec_sum(anl69_best_pe_ratio), 120), std=4), densify(subindustry)), 120)),densify(group_cartesian_product(country, industry)))'}, {'type': 'REGULAR', 'settings': {'instrumentType': 'EQUITY', 'region': 'EUR', 'universe': 'TOP2500', 'delay': 1, 'decay': 1, 'neutralization': 'INDUSTRY', 'truncation': 0.08, 'pasteurization': 'ON', 'unitHandling': 'VE

In [10]:
def check_simulation_progress(session, simulation_progress_url):
    """
    检查批量模拟的进度，获取 children 列表。

    Args:
        simulation_progress_url (str): 批量模拟的进度 URL，例如 "https://api.worldquantbrain.com/simulations/12345"

    Returns:
        list or None: 如果成功返回 children 列表，否则返回 None
    """
    while True:
        try:
            simulation_progress = session.get(simulation_progress_url)
            simulation_progress.raise_for_status()

            # 检查是否包含 Retry-After 头，表示服务端暂时不可用
            if simulation_progress.headers.get("Retry-After", 0) != 0:
                time.sleep(float(simulation_progress.headers["Retry-After"]))
                continue
            # 解析响应，提取 children 和状态
            progress_data = simulation_progress.json()
            children = progress_data.get("children", [])
            if not children:
                print("No children found in simulation progress response.")
                return None

            status = progress_data.get("status")
            print(f"Simulation batch status: {status}, children count: {len(children)}")

            if status != "COMPLETE":
                print("Simulation not complete. Will check again later.")

            return children
        except Exception as e:
            retry_count += 1
            print(f"Error sending batch simulation request. Retry {retry_count} after 5s. Error: {e}")
            time.sleep(5)

            if retry_count > 25:
                print("Retry >10: attempting re-login.")
                session = login()
            if retry_count > 30:
                print("Retry >30: giving up on this batch.")
                return None
    

def process_children(session, children):
    """同步处理 children，获取 alphaId 和 alpha detail，写入文件"""
    print(f"Processing {len(children)} children...")
    if not children:
        print("Empty children list received. Skipping processing.")
        return

    brain_api_url = "https://api.worldquantbrain.com"
    max_retries = 3
    retry_delay = 5
    request_timeout = 30
    alpha_id_list = []

    for child in children:
        print(f"Processing child: {child}")
        if not isinstance(child, str):
            print(f"Invalid child format: {child} (Expected string)")
            continue

        for attempt in range(max_retries):
            try:
                print(f"Fetching child simulation progress: {child} (Attempt {attempt + 1}/{max_retries})")
                child_progress = session.get(f"{brain_api_url}/simulations/{child}", timeout=request_timeout)
                child_progress.raise_for_status()
                child_data = child_progress.json()
                print(f"Child response: {child_data}")

                alpha_id = child_data.get("alpha")
                if not alpha_id:
                    print(f"No alpha_id found for child: {child}. Response: {child_data}")
                    break

                alpha_id_list.append(alpha_id)
                break

            except requests.exceptions.RequestException as e:
                print(f"Error fetching child {child} or alpha detail (Attempt {attempt + 1}/{max_retries}): {e}")
                if attempt < max_retries - 1:
                    print(f"Retrying after {retry_delay}s...")
                    time.sleep(retry_delay)
                else:
                    print(f"Max retries reached for child {child}. Skipping...")
                    if "401" in str(e) or "403" in str(e):
                        print("Authentication error detected. Re-signing in...")
                        session = login()
                    break
            except ValueError as e:
                print(f"Invalid JSON response for child {child}: {e}")
                break
            except Exception as e:
                print(f"Unexpected error processing child {child}: {e}")
                break
    return alpha_id_list

In [11]:

# 创建一个空的 DataFrame，用于保存多个 alpha 因子的结果信息，包含如下列：
# 'alpha_id'（因子ID）、'neutralization'（中性化方式）、'decay'（衰减方式）、
# 'sharpe'（夏普比率）、'fitness'（适应度）、'turnover'（换手率）、'margin'（保证金需求）
df_list = pd.DataFrame(columns=['alpha_id', 'neutralization', 'decay', 'sharpe', 'fitness', 'turnover', 'margin'])

session = login()
# 调用 locate_alpha 函数，用 alpha_id_ori 获取该 alpha 因子的详细信息
tem = locate_alpha(session, alpha_id_ori)

# 将 locate_alpha 返回的多个值进行解包，分别赋值给不同的变量
[alpha_id_ori, sharpe, turnover, fitness, margin, exp, region, universe, neutralization, decay, truncation] = tem

# 按照 df_list 的列顺序，整理出一行数据
new_row = [alpha_id_ori, neutralization, decay, sharpe, fitness, turnover, margin]

# 将这一行数据添加到 df_list 的末尾
df_list.loc[len(df_list)] = new_row

# 打印当前的 df_list 内容，查看结果是否正确添加
print(df_list)

for wb in wbs:

    # 发送请求
    url = wb

    children = check_simulation_progress(session, url)
    if not children:
        print(f"No children found in simulation progress response: {url}.")
        continue
    alpha_id_list = process_children(session, children)

    for alpha_id in alpha_id_list:
        tem=locate_alpha(session, alpha_id)
        [alpha_id,  sharpe, turnover, fitness, margin, exp,region,universe,neutralization,decay,truncation]=tem
        new_row=[alpha_id,neutralization,decay,sharpe,fitness,turnover,margin]
        df_list.loc[len(df_list)] = new_row  # 直接赋值（确保 df 已初始化列名）
# 删除 df_list 中 alpha_id 列重复的行，只保留第一次出现的那一行
# subset="alpha_id" 表示根据 "alpha_id" 列判断是否重复
# keep="first" 表示如果有重复值，仅保留第一次出现的那一行，其余删除
df_list = df_list.drop_duplicates(subset="alpha_id", keep="first")

# 打印去重后的 df_list，查看结果（你这行写成 `df_list` 实际不会打印，建议加上 print）
print(df_list)


b'{"user":{"id":"YT33830"},"token":{"expiry":14400.0},"permissions":["BEFORE_AND_AFTER_PERFORMANCE_V2","BRAIN_LABS","BRAIN_LABS_JUPYTER_LAB","CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","SUPER_ALPHA","VISUALIZATION","WORKDAY"]}'
  alpha_id neutralization  decay  sharpe  fitness  turnover    margin
0  dJ8RAGx       INDUSTRY      6    2.19     1.32    0.1276  0.000728
Simulation batch status: COMPLETE, children count: 10
Processing 10 children...
Processing child: 1CpMuYdFB4kZa9DQ3xsoY8k
Fetching child simulation progress: 1CpMuYdFB4kZa9DQ3xsoY8k (Attempt 1/3)
Child response: {'id': '1CpMuYdFB4kZa9DQ3xsoY8k', 'parent': '3XNv25c4E4yJaqlBJRbv16Y', 'type': 'REGULAR', 'settings': {'instrumentType': 'EQUITY', 'region': 'EUR', 'universe': 'TOP2500', 'delay': 1, 'decay': 1, 'neutralization': 'SUBINDUSTRY', 'truncation': 0.08, 'pasteurization': 'ON', 'unitHandling': 'VERIFY', 'nanHandling': 'ON', 'maxTrade': 'OFF', 'language': 'FASTEXPR', 'visualization': False}, 'regular': 'group_zs

In [8]:
from time import sleep
def get_pnl(s, alpha_id):
    finished = False
    while True:
        pnl = s.get('https://api.worldquantbrain.com/alphas/' + alpha_id + '/recordsets/pnl')
        if pnl.headers.get('Retry-After', 0) == 0:
             break
        # print('Sleeping for ' + pnl.headers['Retry-After'] + ' seconds')
        sleep(float(pnl.headers['Retry-After']))
    # print('PNL retrieved')
    return pnl
s 
df1 = pd.DataFrame()

for alpha_id in df_list['alpha_id'].unique():
    print(alpha_id)
    json_data = get_pnl(s, alpha_id).json()['records']
    df = pd.DataFrame(json_data)
    df=df.iloc[:,0:2]
    df.columns = ['date', alpha_id]
    df.set_index('date', inplace=True)
    df1 = pd.merge(df1, df, left_index=True, right_index=True, how='outer')


df1.index = pd.to_datetime(df1.index)
start_time=df1.index[0]-pd.Timedelta(days=1)
start_time

df1

GK5g5AG
wWZWPNY
AENELPw
x5Wb9vg
L3rZqmM
3J5V3Oz
KZQr6Kl
5Yeg90X
AEmvbYR
jOx8Xne
r8VeLPd
oZrPZr6
ZNMXNvj
9kmGknV
MVZg2PL
OgV62Yb
nrQ6XVw
10PG2vM
MVZg2qn
gLdr0ol
XMeNOKz


,GK5g5AG,wWZWPNY,AENELPw,x5Wb9vg,L3rZqmM,3J5V3Oz,KZQr6Kl,5Yeg90X,AEmvbYR,jOx8Xne,...,oZrPZr6,ZNMXNvj,9kmGknV,MVZg2PL,OgV62Yb,nrQ6XVw,10PG2vM,MVZg2qn,gLdr0ol,XMeNOKz
date,,,,,,,,,,,,,,,,,,,,,
2013-01-22,843.0,5821.0,2035.0,443.0,-1593.0,-1270.0,-1101.0,-206.0,-1593.0,-473.0,...,-3225.0,-2047.0,-1946.0,-3682.0,-2613.0,-3791.0,-2955.0,-3682.0,-3473.0,-4102.0
2013-01-23,3099.0,-3357.0,24894.0,18724.0,23601.0,21167.0,13848.0,26113.0,19704.0,13552.0,...,-16035.0,-11396.0,-7564.0,2124.0,1252.0,424.0,3512.0,-313.0,242.0,-6434.0
2013-01-24,11790.0,-9261.0,14212.0,4678.0,7920.0,4691.0,3855.0,10626.0,10038.0,2371.0,...,-2646.0,-7170.0,-12470.0,3994.0,80.0,2247.0,5445.0,-6154.0,-306.0,-2411.0
2013-01-25,41935.0,8508.0,31772.0,17702.0,18422.0,12529.0,9712.0,23010.0,6685.0,9724.0,...,28781.0,22847.0,13354.0,32129.0,22151.0,15623.0,33788.0,8077.0,14944.0,14519.0
2013-01-28,18761.0,-7003.0,28600.0,9229.0,11723.0,-3289.0,2338.0,23485.0,13396.0,884.0,...,-2389.0,14066.0,4472.0,29325.0,12239.0,2057.0,38626.0,14305.0,-1510.0,6461.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-13,37806791.0,38239415.0,35000218.0,34026250.0,33640884.0,30667678.0,16792931.0,30357095.0,17959827.0,16950791.0,...,37433557.0,34048033.0,33086872.0,32736560.0,29691885.0,16434678.0,29582624.0,17371760.0,16418611.0,26516953.0
2023-01-17,37756872.0,38186403.0,34947231.0,33978583.0,33589792.0,30610998.0,16777146.0,30315207.0,17919590.0,16921244.0,...,37384003.0,33995598.0,33038542.0,32684617.0,29631976.0,16414841.0,29537499.0,17339467.0,16386057.0,26486618.0
2023-01-18,37604038.0,38022214.0,34818897.0,33857298.0,33474985.0,30454642.0,16724521.0,30171757.0,17865521.0,16871869.0,...,37249707.0,33882026.0,32932165.0,32584252.0,29493345.0,16378836.0,29409364.0,17297691.0,16352935.0,26404221.0


In [5]:
from time import sleep
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def get_pnl(s, alpha_id):
    while True:
        pnl = s.get('https://api.worldquantbrain.com/alphas/' + alpha_id + '/recordsets/pnl')
        if pnl.headers.get('Retry-After', 0) == 0:
            break
        sleep(float(pnl.headers['Retry-After']))
    return pnl

# 初始化一个空的DataFrame来存储所有PNL数据
df1 = pd.DataFrame()

# 获取所有alpha_id的PNL数据
for alpha_id in df_list['alpha_id'].unique():
    print(f"正在获取 alpha_id: {alpha_id}")
    try:
        json_data = get_pnl(s, alpha_id).json()['records']
        df = pd.DataFrame(json_data)
        df = df.iloc[:, 0:2]
        df.columns = ['date', alpha_id]
        df.set_index('date', inplace=True)
        df1 = pd.merge(df1, df, left_index=True, right_index=True, how='outer')
    except Exception as e:
        print(f"获取 {alpha_id} 时出错: {e}")

# 确保索引是datetime格式
df1.index = pd.to_datetime(df1.index)

# 创建图表 - 使用更简单的方法避免布局错误
fig, ax = plt.subplots(figsize=(14, 8))

# 为每条曲线设置不同的颜色
colors = plt.cm.tab20(np.linspace(0, 1, min(20, len(df1.columns))))

# 绘制所有alpha_id的PNL曲线
for i, alpha_id in enumerate(df1.columns):
    # 计算累积收益
    if df1[alpha_id].notna().any():  # 确保有有效数据
        cumulative_returns = (1 + df1[alpha_id].fillna(0)).cumprod() - 1
        
        # 使用循环颜色
        color = colors[i % len(colors)]
        
        # 绘制曲线
        ax.plot(cumulative_returns.index, cumulative_returns.values, 
                label=alpha_id, color=color, linewidth=1.5, alpha=0.7)

# 设置图表标题和标签
ax.set_title('所有Alpha策略的累积收益曲线', fontsize=16, fontweight='bold')
ax.set_xlabel('日期', fontsize=12)
ax.set_ylabel('累积收益', fontsize=12)

# 添加网格
ax.grid(True, alpha=0.3)

# 旋转x轴标签
plt.setp(ax.get_xticklabels(), rotation=45)

# 处理图例 - 如果alpha太多，使用更聪明的方法
if len(df1.columns) <= 15:
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
else:
    # 对于大量alpha，不显示图例但添加说明
    ax.text(0.02, 0.98, f'共 {len(df1.columns)} 个Alpha策略', 
            transform=ax.transAxes, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# 使用手动调整布局而不是tight_layout
plt.subplots_adjust(left=0.1, right=0.85, bottom=0.15, top=0.9)

# 显示图表
plt.show()

# 显示数据的基本统计信息
print("\n数据统计信息:")
print(f"时间范围: {df1.index.min()} 到 {df1.index.max()}")
print(f"Alpha数量: {len(df1.columns)}")
print(f"数据点数量: {len(df1)}")

# 可选：显示前几个alpha的统计信息
if len(df1.columns) > 0:
    print("\n前5个Alpha的最终累积收益:")
    for alpha_id in df1.columns[:5]:
        final_return = (1 + df1[alpha_id].fillna(0)).cumprod().iloc[-1] - 1
        print(f"{alpha_id}: {final_return:.2%}")

正在获取 alpha_id: w7MeZY6
正在获取 alpha_id: 131WlJM
正在获取 alpha_id: PLq5Ypp
正在获取 alpha_id: q2v7Yl2
正在获取 alpha_id: nGYaeNx
正在获取 alpha_id: Rxqel7d
正在获取 alpha_id: 3m30k9O
正在获取 alpha_id: EnYQ8pP
正在获取 alpha_id: jkWaP8e
正在获取 alpha_id: JaY8WPn
正在获取 alpha_id: OdqWMAq
正在获取 alpha_id: 131G9nK
正在获取 alpha_id: Odq6YbR
正在获取 alpha_id: lwkgq7l
正在获取 alpha_id: ZVqX5r3
正在获取 alpha_id: amMRYoR
正在获取 alpha_id: 131G9gm
正在获取 alpha_id: w7ONmev
正在获取 alpha_id: PLqPWp7
正在获取 alpha_id: ojAPpl5
正在获取 alpha_id: KAqV09z
正在获取 alpha_id: 95bGq3x
正在获取 alpha_id: QRqWPpW
正在获取 alpha_id: z7QAq7d
正在获取 alpha_id: 2VQGrgw
正在获取 alpha_id: VQqMkWY
正在获取 alpha_id: ZVqXYOx
正在获取 alpha_id: AXYLjRR
正在获取 alpha_id: MvqgMjn
正在获取 alpha_id: AXYLjjY


/Users/sujianan/anaconda/envs/py311/lib/python3.11/site-packages/numpy/core/fromnumeric.py:57: RuntimeWarning: overflow encountered in accumulate


ValueError: arange: cannot compute length

<Figure size 1400x800 with 1 Axes>


数据统计信息:
时间范围: 2013-01-21 00:00:00 到 2023-01-20 00:00:00
Alpha数量: 30
数据点数量: 2581

前5个Alpha的最终累积收益:
w7MeZY6: inf%
131WlJM: inf%
PLq5Ypp: inf%
q2v7Yl2: inf%
nGYaeNx: inf%
